# RMSE calculation - 6 hourly FuXi

* This notebook shows how RMSEs were computed for 6 hourly FuXi outputs.

* The notebook runs with `verif_config_6h.yml` in the same folder.

In [1]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

### Load config

In [2]:
config_name = os.path.realpath('verif_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### Model and lead time to verify

In [3]:
model_name = 'fuxi'
lead_range = conf[model_name]['lead_range']
verif_lead_range = conf[model_name]['verif_lead_range']

leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
leads_verif = list(np.arange(verif_lead_range[0], verif_lead_range[-1]+verif_lead_range[0], verif_lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, leads_verif)

print('Verifying lead times: {}'.format(leads_verif))
print('Verifying lead indices: {}'.format(ind_lead))

Verifying lead times: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
Verifying lead indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [4]:
verif_ind_start = 0; verif_ind_end = 2*(365+366+365)
path_verif = conf[model_name]['save_loc_verif']+'combined_rmse_{:04d}_{:04d}_{:03d}h_{:03d}h_{}.nc'.format(
    verif_ind_start, verif_ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)

## RMSE compute

In [5]:
# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')
    
# Select the specified variables and their levels
variables_levels = conf['ERA5_ours']['verif_variables']

# subset merged ERA5 and unify coord names
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]
# filename_OURS = [fn for fn in filename_OURS if '00Z' in fn]

L_max = len(filename_OURS)
print('Total number of initialization times: {}'.format(L_max))
assert verif_ind_end <= L_max, 'verified indices (days) exceeds the max index available'

filename_OURS = filename_OURS[verif_ind_start:verif_ind_end]

# latitude weighting
lat = xr.open_dataset(filename_OURS[0])["latitude"]
w_lat = np.cos(np.deg2rad(lat))
w_lat = w_lat / w_lat.mean()

# ---------------------------------------------------------------------------------------- #
# RMSE compute
verif_results = []

for fn_ours in filename_OURS:
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variables_levels)
    ds_ours = ds_ours.isel(time=ind_lead)
    ds_ours = ds_ours.compute()
    
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time']).compute()
    
    # RMSE with latitude-based cosine weighting (check w_lat)
    RMSE = np.sqrt(
        (w_lat * (ds_ours - ds_target)**2).mean(['latitude', 'longitude'])
    )
    
    verif_results.append(RMSE.drop_vars('time'))
    
# Combine verif results
ds_verif = xr.concat(verif_results, dim='days')

# # Save the combined dataset
# print('Save to {}'.format(path_verif))
# # ds_verif.to_netcdf(path_verif)

Total number of initialization times: 2192


In [6]:
ds_verif['Z500']

<xarray.DataArray 'Z500' (days: 3, time: 40)>
array([[ 38.822053  ,  56.86419957,  63.06211259,  77.07499892,
         86.69954926, 102.53506345, 108.80005401, 120.26460997,
        130.65041869, 141.70111494, 152.1105897 , 166.3919308 ,
        178.08395473, 194.55784001, 209.92619961, 231.72592184,
        256.32725319, 281.76942641, 306.81836858, 325.48676917,
        349.24897444, 376.91227218, 410.08344004, 442.48283702,
        477.50583014, 507.50796747, 528.2708021 , 537.51868273,
        546.59774789, 552.64379459, 558.10340191, 570.02313642,
        592.64079181, 629.29803463, 667.72311348, 700.11397772,
        719.67323146, 730.44100287, 734.5610913 , 742.47922271],
       [ 35.13033879,  50.00555149,  60.89690681,  76.36259344,
         82.209884  ,  93.89557778, 104.50154548, 115.7792697 ,
        126.19934424, 138.5565641 , 150.62880943, 169.50292351,
        185.61289405, 205.50209513, 225.81074264, 250.89762672,
        280.27564232, 304.94963881, 328.8122307 , 351.74337334,
        377.46442031, 403.8541988 , 430.46792202, 450.76081508,
        465.68259745, 478.87958522, 497.05376058, 515.3987407 ,
        533.61830277, 551.5744886 , 573.76758549, 605.79308688,
        641.43815279, 678.67510574, 709.90925753, 740.72648602,
        762.13336679, 780.26990544, 788.02481864, 795.15147814],
       [ 36.96099701,  54.50019018,  61.14182572,  73.50984167,
         83.57849132,  94.9095595 , 104.39291177, 116.26324761,
        126.27238589, 143.82673992, 156.17242682, 172.23462154,
        187.91040844, 206.44559237, 232.80046036, 259.4588923 ,
        286.67850982, 312.26493621, 337.64937704, 362.66504558,
        386.88420673, 403.22216695, 410.05586284, 415.20398953,
        420.26544998, 429.06204323, 442.6496564 , 463.78178377,
        491.25843764, 524.36897901, 562.96338162, 597.95677053,
        625.37157623, 649.08546847, 670.06566798, 696.54185667,
        717.31411387, 743.18385433, 756.47388999, 765.51034814]])
Dimensions without coordinates: days, time